![manufacturing gears](manufacturing.jpg)

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step, and keeping a close eye on the process is important.

For this project, you're supporting a team that wants to improve how they monitor and control a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process works well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL defines the highest acceptable height for the parts, while the LCL defines the lowest acceptable height for the parts. Ideally, parts should fall between the two limits.

Using SQL window functions and nested queries, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

In [21]:
SELECT submain.*, (CASE WHEN submain.height > submain.LCL AND submain.height < submain.UCL THEN False
ELSE True END) as alert
FROM 
	(SELECT sub.*, sub.avg_height+(3*sub.stddev_height/sqrt(5)) as UCL, 			sub.avg_height-(3*sub.stddev_height/sqrt(5)) as LCL 
	FROM
		(SELECT operator, ROW_NUMBER() OVER(PARTITION BY operator ORDER BY 				item_no) AS ROW_Number, height,
		AVG(height) OVER (PARTITION BY operator ORDER BY item_no ASC ROWS BETWEEN 		  4 PRECEDING AND CURRENT ROW) as avg_height,
		STDDEV(height) OVER (PARTITION BY operator ORDER BY item_no ASC ROWS 			BETWEEN 4 PRECEDING AND CURRENT ROW) as stddev_height FROM 						manufacturing_parts) as sub) as submain
WHERE row_number>=5

operator  row_number  height  ...        ucl        lcl  alert
0       Op-1           5   19.46  ...  21.203912  18.352088  False
1       Op-1           6   20.36  ...  21.375477  18.448523  False
2       Op-1           7   20.22  ...  21.485108  18.574892  False
3       Op-1           8   21.03  ...  21.183369  18.684631  False
4       Op-1           9   19.78  ...  20.973418  19.366582  False
..       ...         ...     ...  ...        ...        ...    ...
415     Op-9          31   19.01  ...  19.176422  18.631578  False
416     Op-9          32   18.57  ...  19.199759  18.528241  False
417     Op-9          33   20.91  ...  20.543613  17.988387   True
418     Op-9          34   21.24  ...  21.410208  17.945792  False
419     Op-9          35   21.66  ...  22.146675  18.409325  False

[420 rows x 8 columns]